In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models

import os
import torch
import librosa

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from resnet import resnet20
from datasets import AudioDataset

In [2]:
# path to urban sound 8k
data_root = "/home/tiz007/228/228_data/UrbanSound8K/"
# path to label
label_path = "/home/tiz007/228/228_data/UrbanSound8K/metadata/UrbanSound8K.csv"  

In [3]:
def lr_schedule(epoch, init_lr):
    if epoch <20:
        return init_lr
    elif epoch>=20 and epoch<40:
        return init_lr/10
    elif epoch>=40 and epoch <80:
        return init_lr/100
    else:
        return init_lr/1000

In [4]:
def train(MAX_EPOCH = 100):

    # initialize dataset (feature can be "mfcc", "spec", "mel_raw")
    audio_dataset = AudioDataset(1, DataRoot=data_root, LabelPath=label_path, feature="spec", mode="train")

    # define lstm model
    cnn_model = models.resnet18(num_classes=10)

    # to gpu
    cnn_model = cnn_model.cuda()
    #print(cnn_model)
    # initialize dataloader
    data_loader = torch.utils.data.DataLoader(audio_dataset, batch_size=32, shuffle=True, num_workers=1)

    # loss function
    loss_fn = nn.CrossEntropyLoss().cuda()

    # lr
    learning_rate = 1e-3

    # initialize optimizer
    optimizer = torch.optim.Adam(cnn_model.parameters(), lr=learning_rate)

    
    for epoch in range(MAX_EPOCH):
        # iterate through dataset
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_schedule(epoch, learning_rate)
            
        # initialize epoch stat
        correct_num = 0
        total_num = 0
        loss_sum = 0

        for idx, data in enumerate(data_loader):

            train_data, labels = data
            
            #train_data = train_data.type(torch.float32)/255
            
            # data to gpu
            train_data = train_data.cuda()
            labels = labels.cuda()

            prob = cnn_model(train_data)   
            loss = loss_fn(prob, labels)

            output = prob.argmax(1)
            loss_sum += loss.item()

            correct_num += (output==labels).sum().double()
            total_num += float(labels.shape[0])

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

        print("epoch acc: {:.4} avg loss: {:.4f}".format(correct_num/total_num, loss_sum/total_num))
        
        if epoch%5 == 4:
            test(cnn_model)

In [5]:
def test(model, ):
    test_dataset = AudioDataset(3, DataRoot=data_root, LabelPath=label_path, feature="spec", mode="test")
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=1)
    
    model.eval()

    correct_num = 0
    total_num = 0
    for idx, data in enumerate(test_loader):
        test_data, labels = data
        test_data = test_data.cuda()
        labels = labels.cuda()

        test_data = test_data.cuda()

        prob = model(test_data)   

        output = prob.argmax(1)
        correct_num += (output==labels).sum().double()
        total_num += float(labels.shape[0])
    
    model.train()

    print("##Testing## epoch acc: {:.4}".format(correct_num/total_num))

In [ ]:
train(200)

verify spec feature success
epoch acc: 0.4646 avg loss: 0.0463
epoch acc: 0.657 avg loss: 0.0303
epoch acc: 0.7148 avg loss: 0.0252
epoch acc: 0.7654 avg loss: 0.0212
epoch acc: 0.8004 avg loss: 0.0182
verify spec feature success
##Testing## epoch acc: 0.7589
epoch acc: 0.8238 avg loss: 0.0161
epoch acc: 0.8406 avg loss: 0.0145
epoch acc: 0.8626 avg loss: 0.0124
epoch acc: 0.8762 avg loss: 0.0114
epoch acc: 0.8843 avg loss: 0.0106
verify spec feature success
##Testing## epoch acc: 0.8486
epoch acc: 0.8922 avg loss: 0.0099
epoch acc: 0.9033 avg loss: 0.0087
epoch acc: 0.9168 avg loss: 0.0078
epoch acc: 0.9113 avg loss: 0.0079
epoch acc: 0.9231 avg loss: 0.0069
verify spec feature success
##Testing## epoch acc: 0.8995
epoch acc: 0.9245 avg loss: 0.0069
epoch acc: 0.9323 avg loss: 0.0061
epoch acc: 0.9397 avg loss: 0.0055
epoch acc: 0.9405 avg loss: 0.0054
epoch acc: 0.9398 avg loss: 0.0055
verify spec feature success
##Testing## epoch acc: 0.9027
epoch acc: 0.9709 avg loss: 0.0028
epoch 